# API challenge

In [1]:
import requests as rq
import plotly.graph_objects as go
import urllib.parse

## Forest sector's contribution to GDP

First an analysis on the forestry sector's contribution to Estonian and Finnish s GDP over 10 years is concluded. Yearly GDP is considered, since Finnish databases do not have quarterly data. GDP and the value of the forestry industry of both countries from 2009 to 2022 is requested from national statistics databases. All monetary values are represented in today's prices.

Base URL-s are defined:

In [2]:
est_base_url = 'https://andmed.stat.ee/api/v1/et/stat/'
fin_base_url = 'https://pxdata.stat.fi:443/PxWeb/api/v1/en/StatFin/'
fin_forest_base_url = 'https://statdb.luke.fi:443/PxWeb/api/v1/en/LUKE/'

Classes for abstracting querys:

In [3]:
class QueryDimension:
    def __init__(self, dimension: str, values: list):
        self.dimension = dimension
        self.values = values
    def get(self) -> dict:
        query_dimension_prepared = {
            "code": self.dimension,
            "selection": {
                "filter": "item",
                "values": [str(value) for value in self.values]
            }
        }
        return query_dimension_prepared

class Query:
    formats = {
        "json": "json-stat2"    }
    def __init__(self):
        self.query_dimensions = []
    def add_query_dimension(self, query_dimension: QueryDimension) -> None:
        self.query_dimensions += [query_dimension]
    def get(self, format: str) -> dict:
        if format not in self.formats:
            raise ValueError("unknown format")
        query_prepared = {
            "query": [query_dimension.get() for query_dimension in self.query_dimensions],
            "response": {
                "format": self.formats[format]
            }
        }
        return query_prepared

# Sample run
#
# import requests as rq
#
# query_dimensions = [
#     QueryDimension("Year", [2021, 2022]),
#     QueryDimension("Quarter", [1]),
#     ...
#     QueryDimension("Unit", ["km"])
# ]
#
# query = Query()
# _ = [query.add_query_dimension(dimension) for dimension in query_dimensions]
#
# response = rq.post(url, json=query.get('json'))

Request for Estonia's GDP in million euros:

In [40]:
est_gdp_endpoint = 'RAA0012'

est_gdp_url = urllib.parse.urljoin(est_base_url, est_gdp_endpoint)

# https://andmed.stat.ee/et/stat/majandus__rahvamajanduse-arvepidamine__sisemajanduse-koguprodukt-(skp)__pehilised-rahvamajanduse-arvepidamise-naitajad/RAA0012

year_range = list(range(2009, 2023))

query_dimensions = [
    QueryDimension("Aasta", year_range),
    QueryDimension("Kvartal", [1]), # Value 1 stands for yearly data, use "I", "II", "III", "IV" for quarterly values
    QueryDimension("Sesoonne korrigeerimine", [1]), # Seasonally and working day unadjusted
    QueryDimension("Näitaja", [2]) # GDP chain-linked volume (reference year 2015), million euros, this means all values in 2015 prices
]
est_gdp_query = Query()
_ = [est_gdp_query.add_query_dimension(dimension) for dimension in query_dimensions]

est_gdp = rq.post(est_gdp_url, json=est_gdp_query.get('json'))
est_gdp.raise_for_status() # Raises errors for bad requests
est_gdp_Meur = est_gdp.json()

print(est_gdp_Meur['value']) #Meur = Mega euros = million euros

[14131.9, 14741.1, 16677.3, 17916.7, 18910.8, 20048.2, 20631.4, 21747.9, 23833.6, 25932.2, 27951, 27430, 31169, 36011.1]


The purpose of seasonal adjustment is to remove systematic calendar-related variation associated with the time of the year, that is, seasonal effects.
An adjustment for working days takes into account influences arising from the number of working days. This means giving consideration to lengths of months, numbers of weekdays and public holidays. Figures adjusted for working days are published for industries where variation in the number of working days has a significant impact on a time series.

Request for Finland's GDP in million euros:

In [32]:
fin_gdp_endpoint = 'vtp/statfin_vtp_pxt_11sf.px'

fin_gdp_url = urllib.parse.urljoin(fin_base_url, fin_gdp_endpoint)

# https://pxdata.stat.fi/PxWeb/pxweb/en/StatFin/StatFin__vtp/statfin_vtp_pxt_11sf.px/

year_range = list(range(2009, 2023))

query_dimensions = [
    QueryDimension("Taloustoimi", ["B1GMH"]), # GDP at market prices
    QueryDimension("Vuosi", year_range),
    QueryDimension("Tiedot", ["vv_hinta_2015"]), # Volume series, reference year 2015, this means values in 2015 prices
]
fin_gdp_query = Query()
_ = [fin_gdp_query.add_query_dimension(dimension) for dimension in query_dimensions]

fin_gdp = rq.post(fin_gdp_url, json=fin_gdp_query.get('json'))
fin_gdp.raise_for_status()
fin_gdp_Meur = fin_gdp.json()

print(fin_gdp_Meur['value'])

[181747, 188143, 197998, 201037, 204321, 206897, 211385, 217518, 226301, 233462, 239858, 238038, 250664, 267688]


Request for Estonia's forest sector's value added in million euros:

In [33]:
est_forest_endpoint = 'RAA0042'

est_forest_url = urllib.parse.urljoin(est_base_url, est_forest_endpoint)

# https://andmed.stat.ee/et/stat/majandus__rahvamajanduse-arvepidamine__sisemajanduse-koguprodukt-(skp)__sisemajanduse-koguprodukt-tootmise-meetodil/RAA0042

year_range = list(range(2009, 2023))

query_dimensions = [
    QueryDimension("Tegevusala (EMTAK 2008) /komponent", [3]), # Forestry and logging
    QueryDimension("Aasta", year_range),
    QueryDimension("Kvartal", [1]), # Value 1 stands for yearly data, use "I", "II", "III", "IV" for quarterly values
    QueryDimension("Näitaja", [2]) # Chain-linked volume (reference year 2015), million euros
]
est_forest_va_query = Query()
_ = [est_forest_va_query.add_query_dimension(dimension) for dimension in query_dimensions]

est_forest_va = rq.post(est_forest_url, json=est_forest_va_query.get('json'))
est_forest_va.raise_for_status()
est_forest_va_Meur = est_forest_va.json()

print(est_forest_va_Meur['value'])

[150.2, 209.7, 269.8, 202.5, 220.7, 258, 271, 265.7, 249.2, 292.4, 319.1, 253.8, 313.3, 415.8]


Request for Finland's forest sector's value added in million euros:

In [34]:
fin_forest_endpoint = 'vtp/statfin_vtp_pxt_123h.px'

fin_forest_url = urllib.parse.urljoin(fin_base_url, fin_forest_endpoint)

# https://pxdata.stat.fi/PxWeb/pxweb/en/StatFin/StatFin__vtp/statfin_vtp_pxt_123h.px/

year_range = list(range(2009, 2023))

query_dimensions = [
    QueryDimension("Taloustoimi", ["B1GPH"]), # Gross value added at basic prices
    QueryDimension("Toimiala", ["A02"]), # Forestry and Logging
    QueryDimension("Vuosi", year_range),
    QueryDimension("Tiedot", ["vv_hinta_2015"]), # Volume series, reference year 2015
]
fin_forest_va_query = Query()
_ = [fin_forest_va_query.add_query_dimension(dimension) for dimension in query_dimensions]

fin_forest_va = rq.post(fin_forest_url, json=fin_forest_va_query.get('json'))
fin_forest_va.raise_for_status()
fin_forest_va_Meur = fin_forest_va.json()

print(fin_forest_va_Meur['value'])

[2519, 2833, 3058, 3060, 3306, 3429, 3403, 3582, 3784, 4109, 3925, 4046, 4271, 4361]


Calculations for finding the forest sector's contribution to GDP:

In [8]:
est_forest_to_gdp = [i / j * 100 for i, j in zip(
    est_forest_va_Meur['value'], est_gdp_Meur['value'])]

fin_forest_to_gdp = [i / j * 100 for i, j in zip(
    fin_forest_va_Meur['value'], fin_gdp_Meur['value'])]

The results are plotted using plotly:

In [9]:
# List of years that are being plotted
years = list(range(2009, 2023))

# Traces for Estonian and Finnish forestry sector to GDP data
trace1 = go.Scatter(x=years,
                    y=est_forest_to_gdp,
                    mode='lines+markers',
                    name='Estonia'
                    )

trace2 = go.Scatter(x=years,
                    y=fin_forest_to_gdp,
                    mode='lines+markers',
                    name='Finland'
                    )

# Figure from the two traces
fig = go.Figure(data=[trace1, trace2])

fig.update_layout(
    xaxis=dict(tickmode='array', tickvals=years, ticktext=years),
    title="Forestry sector's contribution to Estonian and Finnish GDP, in 2015 prices",
    xaxis_title='Year',
    yaxis_title="Percentage of forestry sector in GDP"
    )

# Set y-axis to start from 0
max_value = max(max(est_forest_to_gdp), max(fin_forest_to_gdp))
fig.update_yaxes(range=[0, (max_value + 0.1)])

# Showing the figure
fig.show()

## Value added per forest unit area

In order to better evaluate which country's forestry sector is more advanced, value added per forest unit area could be considered instead of forestry sector's contribution to GDP. Data from 2009 to 2022 is analysed. Strictly protected areas of forestry land are excluded from the analysis because they do not contribute to value added.

Area of forestry land in Estonia in 1000 ha:

In [35]:
est_forest_area_endpoint = 'KK51'

est_forest_area_url = urllib.parse.urljoin(est_base_url, est_forest_area_endpoint)

# https://andmed.stat.ee/en/stat/keskkond__loodusvarad-ja-nende-kasutamine__metsavaru/KK51

year_range = list(range(2009, 2023))

query_dimensions = [
    QueryDimension("Näitaja", [1]), # Area of forest and other wooded land, 1000 ha
    QueryDimension("Aasta", year_range)
]
est_forest_area_query = Query()
_ = [est_forest_area_query.add_query_dimension(dimension) for dimension in query_dimensions]

est_forest_area = rq.post(est_forest_area_url, json=est_forest_area_query.get('json'))
est_forest_area.raise_for_status()
est_forest_area_Kha = est_forest_area.json()

print(est_forest_area_Kha['value']) # Kha = Kilohectare = 1000 hectares

[2216.8, 2222.4, 2235.3, 2249.7, 2268.4, 2295.3, 2309.9, 2313, 2330.8, 2331, 2332.9, 2325.3, 2325.6, 2325]


Area of forestry land in Finland in 1000 ha:

In [36]:
fin_forest_area_endpoint = '04 Metsa/06 Metsavarat/1.01_Metsatalousmaa.px'

fin_forest_area_url = urllib.parse.urljoin(fin_forest_base_url, fin_forest_area_endpoint)

# https://statdb.luke.fi/PxWeb/pxweb/en/LUKE/LUKE__04%20Metsa__06%20Metsavarat/1.01_Metsatalousmaa.px/

query_dimensions = [
    QueryDimension("inventointi", ["11", "12", "13"]), # Values for year classes
    QueryDimension("maakunta", ["1"]), # Region: whole country
    QueryDimension("maaluokka", ["Metsätalousmaa - Yhteensä"]) # Forestry land: total
]
fin_forest_area_query = Query()
_ = [fin_forest_area_query.add_query_dimension(dimension) for dimension in query_dimensions]

fin_forest_area = rq.post(fin_forest_area_url, json=fin_forest_area_query.get('json'))
fin_forest_area.raise_for_status()
fin_forest_area_Kha = fin_forest_area.json()

print('For 2009-2013, 2014-2018, 2018-2022:')
print(fin_forest_area_Kha['value'])

For 2009-2013, 2014-2018, 2018-2022:
[26192, 26246, 26229]


Percentage of forestry land that is strictly protected in Estonia:

In [37]:
est_forest_protected_endpoint = 'SN11'

est_forest_protected_url = urllib.parse.urljoin(est_base_url, est_forest_protected_endpoint)

# https://andmed.stat.ee/en/stat/eri-valdkondade-statistika__saastev-areng/SN11

year_range = list(range(2009, 2023))

query_dimensions = [
    QueryDimension("Näitaja", ["142"]), # Share of strictly protected areas in forest land
    QueryDimension("Aasta", year_range)
]
est_forest_protected_query = Query()
_ = [est_forest_protected_query.add_query_dimension(dimension) for dimension in query_dimensions]

est_forest_protected = rq.post(est_forest_protected_url, json=est_forest_protected_query.get('json'))
est_forest_protected.raise_for_status()
est_forest_protected_percent = est_forest_protected.json()

print(est_forest_protected_percent['value'])

[10, 9.6, 10.1, 10.8, 11.4, 11.3, 12.3, 12.9, 13.1, 13.2, 14.1, 14.2, 17.6, 18.1]


Percentage of forestry land that is strictly protected in Finland:

In [38]:
fin_forest_protected_endpoint = '04 Metsa/02 Rakenne ja tuotanto/04 Metsien suojelu/01_metsien-suojelu.px'

fin_forest_protected_url = urllib.parse.urljoin(fin_forest_base_url, fin_forest_protected_endpoint)

# https://statdb.luke.fi/PxWeb/pxweb/en/LUKE/LUKE__04%20Metsa__02%20Rakenne%20ja%20tuotanto__04%20Metsien%20suojelu/01_metsien-suojelu.px/

query_dimensions = [
    QueryDimension("suojeluluokka", ["1-2YHT"]), # Protection categorty: protected forests, total
    QueryDimension("maaluokka", ["MAA_YHT"]), # Land class: forestry land, total
    QueryDimension("pinta-ala ja osuus", ["%"]) # Percentage of total forestry area
]
fin_forest_protected_query = Query()
_ = [fin_forest_protected_query.add_query_dimension(dimension) for dimension in query_dimensions]

fin_forest_protected = rq.post(fin_forest_protected_url,
                               json=fin_forest_protected_query.get('json'))
fin_forest_protected.raise_for_status()
fin_forest_protected_percent = fin_forest_protected.json()

print('For corresponding values of Finnish forestry area:')
print(fin_forest_protected_percent['value'])

For corresponding values of Finnish forestry area:
[4499.7, 4711, 4783.3]


Computations for removing protected areas from total forestry land

In [14]:
# Estonian unprotected forests, protected areas were given as percentages
usable_forestry_area_est = [i * (100 - j) / 100 for i, j in zip(
    est_forest_area_Kha['value'], est_forest_protected_percent['value'])]

# Finnish unprotected forests, protected areas were given as area (1000 ha)
usable_forestry_area_fin = [i * (100 - j) / 100 for i, j in zip(
    fin_forest_area_Kha['value'], fin_forest_protected_percent['value'])]

# Expanding the Finnish list so there is a value for every year
usable_forestry_area_fin_expanded = (
    usable_forestry_area_fin[0:1] * 5 +
    usable_forestry_area_fin[1:2] * 5 +
    usable_forestry_area_fin[2:3] * 4
    )

Calculating value added per 1 ha of unprotected forestry area

In [15]:
value_added_per_area_est = [(i * 1e6) / (j * 1e3) for i, j in zip(
    est_forest_va_Meur['value'], usable_forestry_area_est)]

value_added_per_area_fin = [(i * 1e6) / (j * 1e3) for i, j in zip(
    fin_forest_va_Meur['value'], usable_forestry_area_fin_expanded)]

The results are plotted:

In [60]:
# List of years that are being plotted
years = list(range(2009, 2023))

# Traces for Estonian and Finnish value added per forest unit area
trace_est = go.Scatter(x=years,
                       y=value_added_per_area_est,
                       mode='lines+markers',
                       name='Estonia'
                       )

trace_fin = go.Scatter(x=years,
                       y=value_added_per_area_fin,
                       mode='lines+markers',
                       name='Finland'
                       )

# Figure from the two traces
fig = go.Figure(data=[trace_est, trace_fin])

fig.update_layout(
    xaxis=dict(tickmode='array', tickvals=years, ticktext=years),
    title="Value added per 1 hectare of forestry area (in euros, 2015 prices) for Estonia and Finland",
    xaxis_title='Year',
    yaxis_title="Value added per 1 ha of forestry area"
)
# Showing the figure
fig.show()

## Alternative plots with Finnish forestry sector's value added from Luke database

The Finnish National Resources Institute (Luke) also has information on the forestry sector's value added and their results differ from the Statistics Finland database. It might be interesting to also look at the results using the values from the Luke database.

In [46]:
luke_forest_va_endpoint = '04 Metsa/08 Muut/Metsateollisuus/10.05_Metsateollisuuden_tuotannon_bruttoarvo_ja.px'

luke_forest_va_url = urllib.parse.urljoin(fin_forest_base_url, luke_forest_va_endpoint)

# https://statdb.luke.fi/PxWeb/pxweb/en/LUKE/LUKE__04%20Metsa__08%20Muut__Metsateollisuus/10.05_Metsateollisuuden_tuotannon_bruttoarvo_ja.px

year_range = list(range(2009, 2022))

query_dimensions = [
    QueryDimension("vuosi", year_range),
    QueryDimension("arvo", ["Tuotannon jalostusarvo"]), # Added value of production, million euros
    QueryDimension("teollisuusala", ["Metsäteollisuus - Yhteensä"]) # Forest industries - Total
]

luke_forest_va_query = Query()
_ = [luke_forest_va_query.add_query_dimension(dimension) for dimension in query_dimensions]

luke_forest_va = rq.post(luke_forest_va_url, json=luke_forest_va_query.get('json'))
luke_forest_va.raise_for_status()
luke_forest_va_Meur = luke_forest_va.json()

print('For years 2009 - 2021')
print(luke_forest_va_Meur['value'])

For years 2009 - 2021
[2581, 4145, 3680, 3533, 3918, 3828, 4205, 4038, 4563, 5001, 4163, 3519, 5487]


In [54]:
alt_fin_forest_to_gdp = [i / j * 100 for i, j in zip(
    luke_forest_va_Meur['value'], fin_gdp_Meur['value'][0:-1])]

In [57]:
# List of years that are being plotted
years = list(range(2009, 2023))

# Traces for Estonian and Finnish forestry sector to GDP data
trace1 = go.Scatter(x=years,
                    y=est_forest_to_gdp,
                    mode='lines+markers',
                    name='Estonia'
                    )

trace2 = go.Scatter(x=years,
                    y=fin_forest_to_gdp,
                    mode='lines+markers',
                    name='Finland from Statistics Finland database'
                    )

trace3 = go.Scatter(x=years,
                    y=alt_fin_forest_to_gdp,
                    mode='lines+markers',
                    name='Finland from Luke database'
                    )

# Figure from the two traces
fig = go.Figure(data=[trace1, trace2, trace3])

fig.update_layout(
    xaxis=dict(tickmode='array', tickvals=years, ticktext=years),
    title="Forestry sector's contribution to Estonian and Finnish GDP, with alternative values from Luke database",
    xaxis_title='Year',
    yaxis_title="Percentage of forestry sector in GDP"
    )

# Set y-axis to start from 0
max_value = max(max(est_forest_to_gdp), max(alt_fin_forest_to_gdp), max(fin_forest_to_gdp))
fig.update_yaxes(range=[0, (max_value + 0.1)])

# Showing the figure
fig.show()

In [58]:
alt_value_added_per_area_fin = [(i * 1e6) / (j * 1e3) for i, j in zip(
    luke_forest_va_Meur['value'], usable_forestry_area_fin_expanded[0:-1])]

In [59]:
# List of years that are being plotted
years = list(range(2009, 2023))

# Traces for Estonian and Finnish value added per forest unit area
trace_est = go.Scatter(x=years,
                       y=value_added_per_area_est,
                       mode='lines+markers',
                       name='Estonia'
                       )

trace_fin = go.Scatter(x=years,
                       y=value_added_per_area_fin,
                       mode='lines+markers',
                       name='Finland (Statistics Finland database)'
                       )

trace_fin2 = go.Scatter(x=years,
                       y=alt_value_added_per_area_fin,
                       mode='lines+markers',
                       name='Finland (Luke database)'
                       )

# Figure from the two traces
fig = go.Figure(data=[trace_est, trace_fin, trace_fin2])

fig.update_layout(
    xaxis=dict(tickmode='array', tickvals=years, ticktext=years),
    title="Value added per 1 hectare of forestry area (in euros) for Estonia and Finland, with alternative values from Luke database",
    xaxis_title='Year',
    yaxis_title="Value added per 1 ha of forestry area"
)
# Showing the figure
fig.show()

There is a noticable difference in the graphs using values for Finnish forestry sector's value added from Statistics Finland database and Luke database.

It is difficult to say exactly where the difference in values comes from since Statistics Finland is listed as a source for the Luke forestry sector's value added table. However for our comparisons it is still reasonable to rather consider the Statistics Finland data, because it is compiled in accordance with EU standards, so it can be compared with other EU states.